In [15]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series", "all_elements"] }
:dep rstrading = { version = "*", path="../" }
:dep csv = { version = "*" }
:dep serde = { version = "1.0.143", features = ["derive"] }
:dep chrono = { version = "0.4.21" }

extern crate plotters;
extern crate rstrading;
extern crate csv;
use std::fs::File;
use serde::{Deserialize}; 

use rstrading::data::Candle;
use chrono::{DateTime, Utc};

In [29]:
/// Load the json data into variables

const DATA_HISTORIC_XAUUSD_CANDLES_1M: &str = "./candles/xauusd_1m.csv";

#[derive(Deserialize)]
struct Record {
    ticker: String,
    date: String,
    time: String,
    open: f64,
    high: f64,
    low: f64,
    close: f64,
    vol: i32,
}

fn load_json_market_event_candles() -> Vec<Candle> {
    let file = File::open(DATA_HISTORIC_XAUUSD_CANDLES_1M).unwrap();
    let mut rdr = csv::ReaderBuilder::new().has_headers(true).from_reader(file);
    let mut candles = Vec::new();
    
    for result in rdr.deserialize::<Record>() {
       
        let record = result.unwrap();
        let date_str = record.date + " " + record.time.as_str() + " +00:00";
        let close_time = DateTime::parse_from_str(&date_str, "%Y%m%d %H%M%S %z").unwrap().with_timezone(&Utc);
        candles.push(Candle{
            close_time,
            open: record.open,
            high: record.high,
            low: record.low,
            close: record.close,
            volume: Some(record.vol as f64),
            trade_count: None,
        })
    }
    
    candles
}

let candles = load_json_market_event_candles();
// candles

[Candle { close_time: 2001-01-02T23:09:00Z, open: 268.8, high: 268.8, low: 268.8, close: 268.8, volume: Some(4.0), trade_count: None }, Candle { close_time: 2001-01-02T23:12:00Z, open: 268.8, high: 268.8, low: 268.8, close: 268.8, volume: Some(4.0), trade_count: None }, Candle { close_time: 2001-01-02T23:16:00Z, open: 268.9, high: 268.9, low: 268.9, close: 268.9, volume: Some(4.0), trade_count: None }, Candle { close_time: 2001-01-02T23:38:00Z, open: 268.8, high: 268.8, low: 268.8, close: 268.8, volume: Some(4.0), trade_count: None }, Candle { close_time: 2001-01-02T23:41:00Z, open: 268.8, high: 268.8, low: 268.8, close: 268.8, volume: Some(4.0), trade_count: None }, Candle { close_time: 2001-01-02T23:50:00Z, open: 268.8, high: 268.8, low: 268.8, close: 268.8, volume: Some(4.0), trade_count: None }, Candle { close_time: 2001-01-02T23:52:00Z, open: 268.6, high: 268.6, low: 268.6, close: 268.6, volume: Some(4.0), trade_count: None }, Candle { close_time: 2001-01-03T00:03:00Z, open: 268.9